In [1]:
from pyquil.quil import *
from pyquil.api import get_qc
from pyquil.gates import *
from pyquil.latex import display, to_latex
from pyquil.simulation.tools import lifted_gate, program_unitary, lifted_gate_matrix

In [2]:
from functions import *

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
qc = get_qc("9q-square-qvm")

UnknownApiError: Unable to round-trip http request to upstream: dial tcp 127.0.0.1:5000: connectex: No connection could be made because the target machine actively refused it.

The server has failed to return a proper response. Please describe the problem
and copy the above message into a GitHub issue at:
    https://github.com/rigetti/pyquil/issues

# Programmable recipe
![alt text](rotation_definition.png "Title")
$$R(\theta, \phi) = R_z(-\frac{\phi}{2})R_y(\theta)R_z(\frac{\phi}{2})$$
![alt text](programmble_decomposition.png "Title")

In [ ]:
# program_unitary(G, n_qubits=2)

$$R(\theta, \phi) = R_z(-\frac{\phi}{2})R_y(\theta)R_z(\frac{\phi}{2})\\
U(2) = R(\theta, \phi) R_z(\phi_z)\\
or\\
U(2) = R_z(\theta) R_y(\phi) R_z(\psi)
$$

In [ ]:
# %%writefile -a functions.py
G = Program( CPHASE01(-np.pi/2, control=0, target=1), CPHASE10(-np.pi/2, control=0, target=1) )

def arbitary_single_qubit_circuit(theta, phi, si, qubit):
    return Program( RZ(si, qubit = qubit), RY(phi, qubit = qubit), RZ(theta, qubit = qubit) )

def r_theta_phi_rotation(theta, phi, qubit):
    return arbitary_single_qubit_circuit( - phi/2, theta, phi/2, qubit)

def give_random_single_qubit_gate(qubit):
    theta, si = np.random.uniform(0,2*np.pi, size = 2)
    
    phi_range = np.linspace(0,np.pi)
    p_phi = np.sin(phi_range) / np.sum(np.sin(phi_range))
    phi = np.random.choice(phi_range, p = p_phi)
    return arbitary_single_qubit_circuit(theta, phi, si, qubit = qubit)

def normalized_abs_angle_dist(angle_range):
    dist = np.pi - np.abs( np.pi - angle_range )
    dist /= np.sum(dist)
    return dist

def give_v_circuit(alpha, beta, delta, qubits = [0,1]):
    prog = Program(G,  r_theta_phi_rotation(alpha, 0, qubit =qubits[0]),
                   r_theta_phi_rotation(3*np.pi/2,0, qubit =qubits[1]), G)
    prog += Program( r_theta_phi_rotation(beta, np.pi/2, qubit = qubits[0]), 
                    r_theta_phi_rotation(3*np.pi/2, delta, qubit = qubits[1]), G)
    return prog

def give_random_two_quibt_circuit(qubits):
    a,b,c,d = [give_random_single_qubit_gate(qubit=qubit) for _ in range(2) for qubit in qubits]
    
    angles_range = np.linspace(0,2*np.pi)
    alpha, beta, delta = np.random.choice(angles_range, p = normalized_abs_angle_dist(angles_range),
                                          size = 3)
    
    prog = Program(a, b )
    prog += give_v_circuit(alpha, beta, delta, qubits = [0,1])
    prog += Program(c, d )
    return prog

In [ ]:
program_unitary(give_random_two_quibt_circuit([0,1]), n_qubits=2)

In [ ]:
sample = give_random_two_quibt_circuit([0,1])
# print( qc.compile( sample ) , len(sample), len(qc.compile( sample )))

# Visualization
## single-qubit

In [ ]:
num_samples = 2021

single_qubit_unitary_samples = [program_unitary(give_random_single_qubit_gate(qubit=0), n_qubits = 1) for _ in range(num_samples)]

In [ ]:
single_zero_state_density_matrix = np.array([[1,0],[0,0]]) #|0><0| state
# What is U|0><0|U^T
single_final_states = np.array( [np.dot( np.dot(u,single_zero_state_density_matrix), u.conj().T ) for u in single_qubit_unitary_samples] )

In [ ]:
# lets see where U|0><0|U^T is likely found
single_qubit_bloch_vectors = np.array([convert_to_bloch_vector(s) for s in single_final_states])

In [ ]:
plot_bloch_sphere(single_qubit_bloch_vectors)

## Two qubits

In [ ]:
num_samples = 2021

two_qubit_unitary_samples = [program_unitary(give_random_two_quibt_circuit([0,1]), n_qubits = 2) for _ in range(num_samples)]

In [ ]:
two_zero_state_density_matrix = np.kron( np.array([[1,0],[0,0]]) , np.array([[1,0],[0,0]]) )
two_final_states = np.array( [np.dot( np.dot(u,two_zero_state_density_matrix), u.conj().T ) for u in two_qubit_unitary_samples] )

In [ ]:
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
I = np.eye(2)

two_qubit_unitary_basis = np.array([ [ np.kron(x,y) for x in [X,Y,Z] ] for y in [X,Y,Z] ])
# two_qubit_unitary_basis[0]

In [ ]:
# Used the mixed state simulator so we could have the density matrix for this part!
def reduce_to_bloch_vector(rho, sigma_arr:np.array):
    """Reduce a density matrix to a Bloch vector."""
    ax = np.trace(np.dot(rho, sigma_arr[0])).real
    ay = np.trace(np.dot(rho, sigma_arr[1])).real
    az = np.trace(np.dot(rho, sigma_arr[2])).real
    return [ax, ay, az]

In [ ]:
two_qubit_bloch_vectors = np.array([reduce_to_bloch_vector(s,two_qubit_unitary_basis[2]) for s in two_final_states])
plot_bloch_sphere(two_qubit_bloch_vectors)

### Verification of Haar distribution
we need to find out whether the distribution of the points is really uniform over the total space. To do so we first compute the radial distance of points and plot their cumulative histogram. If it grows with the power of 3 then we can be sure that the density of points is uniform in the sphere.

In [ ]:
r_bloch_vectors = np.sqrt( two_qubit_bloch_vectors[:,0]**2 + two_qubit_bloch_vectors[:,1]**2 + two_qubit_bloch_vectors[:,2]**2)

In [ ]:
n_bins = 10

fig, ax = plt.subplots()

bins = np.logspace(np.log10(0.1),np.log10(1),n_bins)
# plot the cumulative histogram
pop, bins, hist_ = ax.hist( r_bloch_vectors, bins, histtype='step',
                           cumulative=True, density = True, label='Empirical')
ax.set_yscale('log')
ax.set_xscale('log')
# ax.plot(np.linspace(0,1,10), np.linspace(0,1,10)**slope)
ax.plot(bins, bins**3)
plt.show()


In [ ]:
non_zero_mask = pop != 0
regression_pop = pop[non_zero_mask]
regression_bins = np.array( bins[:-1] ) [non_zero_mask]
exp, intercept = np.polyfit(np.log(regression_bins), np.log(regression_pop), deg = 1)
exp, intercept

In [ ]:
def exponent_of_dist(data):
    n_bins = 100
    bins = np.logspace(np.log10(0.1),np.log10(1),n_bins)
    # plot the cumulative histogram
    pop, bins, hist_ = plt.hist( data, bins, cumulative=True, density = True)
    # extrapolate the exponent
    non_zero_mask = pop != 0
    regression_pop = pop[non_zero_mask]
    regression_bins = np.array( bins[:-1] ) [non_zero_mask]
    exp, intercept = np.polyfit(np.log(regression_bins), np.log(regression_pop), deg = 1)
    plt.close()
    return exp

In [ ]:
exp_list = []
for i in range(len(two_qubit_unitary_basis)):
    two_qubit_bloch_vectors = np.array([reduce_to_bloch_vector(s,two_qubit_unitary_basis[i]) for s in two_final_states])
    r_bloch_vectors = np.sqrt( two_qubit_bloch_vectors[:,0]**2 + two_qubit_bloch_vectors[:,1]**2 + two_qubit_bloch_vectors[:,2]**2)
    exp_list.append(exponent_of_dist(r_bloch_vectors))
exp_list

# Matrix decomposition verification


In [ ]:
from scipy.stats import unitary_group
import cmath

In [ ]:
u_matrix = unitary_group.rvs(4)
lambda_unitary = np.array([ [1, 1j , 0 , 0],[0, 0, 1j, 1],[0, 0, 1j, -1],[1, -1j, 0, 0] ]) / np.sqrt(2)

In [ ]:
def matrix_in_magic_basis(matrix):
    return np.dot( lambda_unitary.conj().transpose(), np.dot(matrix, lambda_unitary) )

def matrix_out_magic_basis(magic_matrix):
    return np.dot( lambda_unitary, np.dot(magic_matrix, lambda_unitary.conj().transpose()) )

def relative_phases(complex_arr:np.array):
    phases = np.array( [cmath.phase(x) for x in complex_arr] )
    phases = np.sort(phases)
    phases -= phases[0]
    return phases

Strip U of any global phase

In [ ]:
u_matrix /= ( np.linalg.det(u_matrix) )**(1/4)

In [ ]:
u_magic_matrix = matrix_in_magic_basis(u_matrix)

In [ ]:
u_u_T = np.dot(u_magic_matrix, u_magic_matrix.transpose())

In [ ]:
u_u_T_eigen_values, u_u_T_eigen_vectors = np.linalg.eig(u_u_T)

In [ ]:
relative_phases(u_u_T_eigen_values)

#### V Gate:

In [ ]:
# eigen_values_phases = [cmath.phase(x) for x in u_u_T_eigen_values]
# alpha, beta, delta = np.array([eigen_values_phases[0] + eigen_values_phases[1],
#                                eigen_values_phases[0] + eigen_values_phases[2],
#                                eigen_values_phases[1] + eigen_values_phases[2] ]) / 2
# v_matrix = program_unitary(give_v_circuit(alpha, beta, delta), n_qubits=2)
# v_matrix

In [ ]:
eigen_values_phases = relative_phases(u_u_T_eigen_values)[1:]
# eigen_values_phases = [cmath.phase(x) for x in u_u_T_eigen_values]
alpha, beta, delta = np.array([eigen_values_phases[0] + eigen_values_phases[1],
                               eigen_values_phases[0] + eigen_values_phases[2],
                               eigen_values_phases[1] + eigen_values_phases[2] ]) / 2
v_matrix = program_unitary(give_v_circuit(alpha, beta, delta), n_qubits=2)
v_matrix

In [ ]:
eigen_values_phases

In [ ]:
v_magic_matrix = matrix_in_magic_basis(v_matrix)
v_v_T = np.dot(v_magic_matrix, v_magic_matrix.transpose())

Check whether vv^T and uu^T have same eigenvalues

In [ ]:
v_v_T_eigen_values, v_v_T_eigen_vectors = np.linalg.eig(v_v_T)

In [ ]:
relative_phases(v_v_T_eigen_values), relative_phases(u_u_T_eigen_values)

#### K and L

In [ ]:
v_v_T_eigen_values, v_v_T_eigen_vectors = np.linalg.eig(v_v_T)

In [ ]:
k_matrix = np.copy(v_v_T_eigen_vectors)
l_matrix = np.copy(u_u_T_eigen_vectors)

#### AB
$$ A \otimes B = \Lambda ( v^\dagger k^T l u ) \Lambda^\dagger $$

In [ ]:
a_tensor_b = matrix_out_magic_basis( np.dot( v_magic_matrix.conjugate().transpose(),
                                            np.dot(k_matrix.transpose(),np.dot(l_matrix, u_magic_matrix))) )
a_tensor_b

#### CD
$$ C \otimes D = \Lambda ( l^T k ) \Lambda^\dagger $$

In [ ]:
c_tensor_d = matrix_in_magic_basis( np.dot(l_matrix.transpose(), k_matrix) )
c_tensor_d

#### Construct U
$$ U = (A \otimes B) V (C \otimes D) $$

In [ ]:
u_constructed = np.dot(a_tensor_b, np.dot(v_matrix, c_tensor_d))
u_constructed

In [ ]:
u_matrix

#### Check Check
Given U and constructed U are allowed to be different by a phase factor. So we should have their dot to be an identity with a global phase shift.
$$ U_g U^{\dagger}_c  = e^{i\phi I}$$

In [ ]:
np.dot(u_matrix, u_constructed.conjugate().transpose())

In [ ]:
# np.linalg.eig(u_matrix)[0], np.linalg.eig(u_constructed)[0]

In [ ]:
# u_constructed_phase = [cmath.phase(x) for x in u_u_T_eigen_values]

In [ ]:
print(qc.compiler.quil_to_native_quil(Program(XY(angle = np.pi,q1= 0, q2 = 1))))

In [ ]:
task = program_unitary(qc.compiler.quil_to_native_quil(Program(XY(angle = np.pi,q1= 0, q2 = 1))), n_qubits=2)

In [ ]:
initial_state = np.array([0,0,1,0])

In [ ]:
np.dot(task, initial_state)